# 

## Load dependences and setting output configuration

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
import pandas as pd
%pylab inline
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))
from mcfly import modelgen, find_architecture
from keras.models import load_model

## Load data from npy files

In [3]:
import pickle
dico = pickle.load(open("F:/1_GTCG/data/19092018/labels_pat1.pickle", "rb"))
window_positions =  dico["id"]
window_size = 200

In [4]:
dico = []


In [5]:
datapath = 'F:/1_GTCG/data/TestData_060818.tar/TestData_060818/Patient1' 
germline_1_datapath = datapath + '/ChannelData/unzipped/'
ger_1_labels_datapath = datapath + '/LabelData/unzipped/'
model_path = os.path.join('F:/CNN/notebooks/',  'Gtcg_13_9')
model_reloaded = load_model(model_path)

In [6]:
def loadAllWindowsAndLabels(i):#i is the chromozome  
    global G1_data_new
    global G1_labels_new
    G1_data_new =np.load(germline_1_datapath + str(i) + '_channel_maker_real_germline.npy/' + str(i) +'_channel_maker_real_germline.npy')
    G1_labels_new = np.load(ger_1_labels_datapath  +  str(i) +'_label_ci_full_overlap.npy/' +str(i) +'_label_ci_full_overlap.npy')  

In [7]:
def normalize(X):       
    for i in range (0, len(X)):                              
        for j in range(0,number_channels):                                                                                            
            X[i][j] = np.divide(X[i][j], coverage)*100 

In [8]:
def eliminate_noise_chanels(X):
    return np.delete(X, [4-1,5-1,6-1,7-1,8-1,9-1,10-1,12-1,13-1,15-1,16-1, 17-1,18-1,19-1,23-1,24-1,26-1,27-1], axis=1)    

In [9]:
def eliminate_distance_chanels(X):
    return np.delete(X, [3,4,7,8], axis=1)

In [10]:
def differentiate(X):
    for i in range(0, X.shape[0]):        
        for j in range(0, X.shape[1]):
            for k in range(0, X.shape[2]-1):
                X[i][j][k] = X[i][j][k+1] - X[i][j][k]
    return X[:,:,:-1]        

In [11]:
def transposeDataset(X):
    image = []
    for i in range (0, len(X -1)):
        tr = X[i].transpose()
        image.append(tr)
    return np.array(image)

In [12]:
def preProcessData(X_test, y_test):       
    global number_channels
    global coverage
    number_channels = X_test.shape[1]
    coverage = np.median(X_test[:, 0, :]) #the median of the coverage in the normal cells
    normalize(X_test)
    X_test = eliminate_noise_chanels(X_test)
    X_test = eliminate_distance_chanels(X_test)
    X_test = differentiate(X_test)
    number_channels = X_test.shape[1]
    print(number_channels)
    image3 = transposeDataset(X_test)
    X_test = image3    
    return X_test
    #n_channels = len(X_test[0][0])
    #mapclasses = {'DEL_start': 1, 'DEL_end': 0,  'noSV': 2, 'UK':3}
    #y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
    #global y_test_binary
    #y_test_binary= to_categorical(y_test)
    #global num_classes
    #num_classes = y_test_binary.shape[1]

In [13]:
def predict(model_reloaded):
    global labels 
    global predicted
    model = model_reloaded 
    datasize = X_test.shape[0]
    probs = model.predict_proba(X_test[:datasize,:,:],batch_size=1)
    mapclasses = {'DEL_end': 0, 'DEL_start': 1, 'noSV': 2}
    dict = mapclasses
    dict_sorted = sorted(dict.items(), key=lambda x: x[1])
    labels = [i[0] for i in dict_sorted]
    predicted = probs.argmax(axis=1)

In [14]:
def savePredictions(chromosome):
    for i in range(0, len(X_test)):
        if predicted[i] in [0,1]:# if it was start or end of a deletion      
            pos = window_positions[str(chromosome)][i]
            outputfile.write(str(pos['chromosome']) + '\t' + str(pos['position']-1) + '\t'+ str(pos['position']) + '\t' +  labels[predicted[i]] + '\n')

In [16]:
for chromosome in range(1,23,1):
    print(chromosome)
    loadAllWindowsAndLabels(chromosome)    
    X_test = G1_data_new
    y_test = G1_labels_new
    X_test = preProcessData(X_test, y_test)   
    predict(model_reloaded)
    outputfile = open("F:/1_GTCG/data/predicted/Patient1", "a")
    savePredictions(chromosome)
    outputfile.close()

1
5
2
5
3
5
4
5
5
5
6
5
7
5
8
5
9
5
10
5
11
5
12
5
13
5
14
5
15
5
16
5
17
5
18
5
19
5
20
5
21
5
22
5


In [17]:
for chromosome in ['X', 'Y', 'MT']:
    print(chromosome)
    loadAllWindowsAndLabels(chromosome)    
    X_test = G1_data_new
    y_test = G1_labels_new
    X_test = preProcessData(X_test, y_test)   
    predict(model_reloaded)
    outputfile = open("F:/1_GTCG/data/predicted/Patient1", "a")
    savePredictions(chromosome)
    outputfile.close()

X
5
Y
5


KeyError: 'Y'

## Inspect model predictions on test data

In [ ]:
y_index = y_test_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)
confusion_matrix

## Test best current model on testset

### Overall accuracy on the test set:

In [ ]:
%%time
score_test = model.evaluate(X_test, y_test_binary, verbose=False)
print('Test loss and accuracy of best model: ' + str(score_test))